In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import matplotlib


In [16]:
browser = Browser('chrome', headless=False)

# Visit the Mars NASA news site
url = 'https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html'
browser.visit(url)
html = browser.html
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)
column_soup = soup(html, 'html.parser')

In [15]:
column = column_soup.find_all('th')

for columns in column:
    print(columns.text)

id
terrestrial_date
sol
ls
month
min_temp
pressure
